# DVIS 3D Scene Visualizer
This notebook is used to visualize the 3D scene data using [DVIS library](https://github.com/SirWyver/dvis). To run it, follow the instructions described in the DVIS repository. 

After installation, don't forget to launch the server as described in the repository!

### DVIS Info

Axes mapping:
* x - red
* y - green (vertical?)
* z - blue

In [ ]:
from dvis import dvis
from mathutils import Matrix
import numpy as np
from scipy.spatial.transform import Rotation
import os
import json

In [ ]:
def encode_rotation(normalized_axes, rotation_angle, rotation_axis):
    # Convert rotation angle to radians
    rotation_angle_rad = np.deg2rad(rotation_angle)

    if rotation_axis == 'x':
        rotation_matrix = np.array([
            [1, 0, 0],
            [0, np.cos(rotation_angle_rad), -np.sin(rotation_angle_rad)],
            [0, np.sin(rotation_angle_rad), np.cos(rotation_angle_rad)]
        ])
    elif rotation_axis == 'y':
        rotation_matrix = np.array([
            [np.cos(rotation_angle_rad), 0, np.sin(rotation_angle_rad)],
            [0, 1, 0],
            [-np.sin(rotation_angle_rad), 0, np.cos(rotation_angle_rad)]
        ])
    elif rotation_axis == 'z':
        rotation_matrix = np.array([
            [np.cos(rotation_angle_rad), -np.sin(rotation_angle_rad), 0],
            [np.sin(rotation_angle_rad), np.cos(rotation_angle_rad), 0],
            [0, 0, 1]
        ])
    else:
        raise ValueError("Invalid rotation axis. Supported values are 'x', 'y', and 'z'.")
    
    encoded_normalized_axes = np.dot(normalized_axes, rotation_matrix)

    return encoded_normalized_axes

def translate_corners(corners, translation):
    translated_corners = corners + translation
    return translated_corners

In [ ]:
# Unit cube definition
unit_cube_corners = np.array([
    [0, 0, 0],
    [0, 0, 1],
    [1, 0, 0],
    [1, 0, 1],
    
    [0, 1, 0],
    [0, 1, 1],
    [1, 1, 0],
    [1, 1, 1],
])

centroid = np.mean(unit_cube_corners, axis=0)
unit_cube_corners = unit_cube_corners - centroid

# Original normalized_axes matrix representing the unit cube's orientation
normalized_axes = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

# Encode degree rotation around the axis
rotation_angle, axis = 0, 'y'
encoded_normalized_axes = encode_rotation(normalized_axes, rotation_angle, axis)

# Apply the encoded rotation to the unit cube corners
unit_cube_corners = np.dot(unit_cube_corners, encoded_normalized_axes)

# Translate the rotated cube
translation = np.array([0, 0, 0])
unit_cube_corners = translate_corners(unit_cube_corners, translation)

dvis(unit_cube_corners, 'corners', c=-1)

In [ ]:
# Specify the path to the dataset folder
dataset_path = 'datasets/data'

# scene_id = 'test/0cac758d-8d6f-2d13-8d21-9133668b4d7d'
scene_id = 'gen/scene_0'

def generate_corners(obj):
    obb = obj['obb']
    axes_lengths = obb['axesLengths']
    centroid = obb['centroid']
    normalized_axes = np.reshape(obb['normalizedAxes'], (3, 3))
    
    axes_lengths = np.array(axes_lengths)
    centroid = np.array(centroid)
    normalized_axes = np.array(normalized_axes)
    
    # Swap y and z axes
    # normalized_axes[[1, 2]] = normalized_axes[[2, 1]] # TODO: rotation is off
    axes_lengths[[1, 2]] = axes_lengths[[2, 1]]
    centroid[[1, 2]] = centroid[[2, 1]]

    corners = np.zeros((8, 3))
    for i in range(8):
        corner = unit_cube_corners[i]
        scaled_corner = corner * axes_lengths
        transformed_corner = np.dot(normalized_axes, scaled_corner)
        corners[i] = transformed_corner + centroid

    return corners

def visualize_scene(scene_id):
    scan_folder_path = os.path.join(dataset_path, scene_id)

    # Check if the folder contains semseg.v2.json file
    semseg_file = os.path.join(scan_folder_path, 'semseg.v2.json')
    if not os.path.isfile(semseg_file):
        exit(1)

    # Read and parse the semseg.v2.json file
    with open(semseg_file, 'r') as file:
        semseg_data = json.load(file)

    scan_id = semseg_data['scan_id']
    seg_groups = semseg_data['segGroups']
    
    unique_dom_normals = {}
    colors_labels_map = {}
    col_index = 0

    for obj in seg_groups:
        # if obj['dominantNormal'][0] != 0:
        #     continue
        
        # if obj['label'] in ['wall', 'floor', 'ceiling']:
        #     continue
        
        corners = generate_corners(obj)
        
        print(obj['label'])
        unique_dom_normals[str(obj['dominantNormal'])] = unique_dom_normals.get(str(obj['dominantNormal']), []) + [obj['label']]
        
        colors_labels_map[obj['label']] = colors_labels_map.get(obj['label'], col_index)
        col_index += 1
        # Pass the corners to the visualizer
        dvis(corners, "corners", name=obj['label'], c=colors_labels_map[obj['label']])
        
        # break
        
    print("Detected unique dominant normals: ", len(unique_dom_normals))
    for dom_normal, labels in unique_dom_normals.items():
        print(dom_normal, labels)

visualize_scene(scene_id)

### DVIS Example from the original reposity

Use it to test the connection to the local server

In [ ]:
# Unit cube corners
corners = np.array([
    [0, 0, 0],
    [0, 1, 0],
    [1, 0, 0],
    [1, 1, 0],
    [0, 0, 1],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 1]
])

dvis(corners, 'corners', name='corners', c=0, s=3, vis_conf={'opacity':0.9})

In [ ]:
# axis-aligned bounding box, (min,max)-coords: (,6) format
dvis(np.array([0,1,2,3,4,5]),'bbox', s=2, c=3)
# batch of 5 bounding boxes
random_bboxes = np.random.randint(0,10,(5,3))
random_bboxes = np.concatenate([random_bboxes, random_bboxes+np.random.randint(0,10,(5,3))],1)
# format (Nx6)
dvis(random_bboxes, 'bbox', name='bbox_batch')

# batch of oriented bounding boxes (z is upwards)
centers = np.random.randint(0,10,(5,3))
extents = np.random.uniform(1,5,(5,3))
orients = np.random.uniform(-np.pi, np.pi, (5,1))
hbboxes = np.concatenate([centers,extents,orients],1)
dvis(hbboxes, 'hbboxes', c=3, name='OBB')
# batch of colored oriented bboxes
colors = np.random.randint(0,30,(5,1))
hbboxes_c = np.concatenate([centers-10,extents,orients,colors],1)
dvis(hbboxes_c, 'hbboxes_c', c=3, name='OBB colored')
# vector (start pos, end pos)
dvis([1,1,1,4,5,6]  ,'vec', name='vec')
# vecs (several vectors )
dvis(np.concatenate([np.zeros((20,3)), np.random.rand(20,3)],1)  ,'vec', name='batched vec')
# line (line connecting points)
dvis(np.array([[0,0,0],[-1,-1,-1],[0,-1,0],[0,-1,-1],[0,0,-1]]) ,'line', name='line', s=5, c=2)
# arrow
dvis(3*np.eye(4) ,'arrow', name='arrow', s=0.4, c=3)
# transformation (displayed as axes helper)
transf = np.eye(4)
transf[:3,3] = np.array([-1,-2,-2])
dvis(transf, 'transform',name='transform')